In [1]:
import numpy as np
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

### Extract All The Links

In [14]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# start at this page
driver.get('https://dialnet.unirioja.es/tesis/fechalectura?inicio=7000')

In [11]:
%%time
absLinks=[]
max_pages = 0
while True:
    link1 = driver.find_elements_by_css_selector("span.titulo")
    for tags in link1:
        link2 = tags.find_elements_by_tag_name("a")
        for link3 in link2:
            absLinks.append(link3.get_attribute("href"))
    
    # extract links
    try:
        time.sleep(7)
        driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
        max_pages += 1
        if max_pages == 170:
            break
    # stop if no more pages available
    except NoSuchElementException:
        break

Wall time: 24min 4s


### Get Contents From Each Link

In [ ]:
data = []
for link in absLinks:
    time.sleep(5)
    driver.get(link)

    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text

    except NoSuchElementException:
        break

    #collect the data
    lang= {'content': language}
    data.append(lang)

In [23]:
en_sp = pd.DataFrame(data)

In [24]:
len(en_sp)

188

### Data Cleaning

In [25]:
# remove \n 
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))

# return only rows with 'Espanol' in the text (because it contains english translate too)
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

# remove rows with 0
en_sp = en_sp[en_sp['Spanish'] != 0]

# create two new variables: Spanish_text, English_text to separate the translations
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])

# remove 'Galego' translation in the English_text variable
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

en_sp['Spanish_text'] = en_sp['Spanish_text'].apply(lambda x: x.split('Español')[-1])

# remove any text that contains 'Español' in the English_text
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: 0 if 'Español' in x else x)

# remove rows with 0
en_sp = en_sp[en_sp['English_text'] != 0]


In [26]:
# make sure Spanish_text and English_text do not have the same value
en_sp = en_sp[en_sp['Spanish_text'] != en_sp['English_text']]
len(en_sp)

44

In [27]:
# format dataframe
en_sp.reset_index(inplace=True)
en_sp.drop(['index', 'content', 'Spanish'], axis=1, inplace=True)

In [28]:
en_sp = en_sp[:30]

In [33]:
en_sp['Spanish_text'][12]

'La técnica más comúnmente empleada en el estudio del cerebro es el uso de modelos animales ex vivo de ratón pero estos modelos no suelen predecir con precisión las reacciones de los medicamentos para tratar a los pacientes humanos. Como solución, se propone el desarrollo de un modelo neuronal in vitro de origen humano que simule la fisiología y funcionalidad del tejido cerebral humano necesario para obtener resultados preclínicos más representativos. Sin embargo, los modelos in vitro existentes están limitados por una reproducibilidad estocástica entre lotes experimentales.Por tanto, el principal desafío de esta investigación se basa en el desarrollo y caracterización de una prueba de concepto para un modelo neuronal in vitro con una reproducibilidad de diferenciación celular mejorada. Para ello, se fabricaron y caracterizaron nuevas membranas planas poliméricas de poli(ε-caprolactona) (PCL) funcionalizadas con nanomateriales basados en grafeno, en particular oxido de grafeno (GO) y o

In [34]:
en_sp['English_text'][12]

'The most commonly employed technique for studying the brain is the use of mouse ex vivo animal models. However, animal models usually fail to accurately predict drug reactions for treating human patients. As a solution, researchers propose the development of an in vitro neural model of human origin with the target of mimicking the physiology and functionality of human brain tissues, as a way to obtain more representative results. Nevertheless, the existing in vitro models are limited by the high stochastic reproducibility from experimental batches.Therefore, the main challenge of this research is based on the development and characterization of the proof of concept for an in vitro neural model with improved reproducible differentiation. To do this, polymeric poly(ε-caprolactone) (PCL) membranes functionalized with graphene based nanomaterials (graphene oxide (GO) and reduced graphene oxide (rGO)) with high porosity were fabricated and characterized to act as cell support in perfusion 

In [35]:
en_sp.to_csv('lang_doc3.csv', index=False)